# Python On Resonance (PyOR)
## Everybody can simulate NMR

Author: Vineeth Thalakottoor 

Email: vineethfrancis.physics@gmail.com

## Tutorial 18 Basis Transformation

## Load Python packages and define path to the source file "PythonOnResonance.py"

In [1]:
pathSource = '/media/HD2/Vineeth/PostDoc_Simulations/Github/PyOR_v1.0/Source'

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys
sys.path.append(pathSource)

import PythonOnResonance as PyOR

import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib notebook
import sympy as sp
from sympy import *

## Generating Spin System

In [3]:
"""
Define Spin quantum numbers of your spins in "Slist1".
Slist1[0] is spin of first particle and Slist1[1] is spin of second particle.
""";

Slist1 = [1/2,1/2]

In [4]:
"""
Define Planck constant equals 1. 
Because NMR spectroscopists are more interested to write Energy in frequency units. 
if False then hbarEQ1 = hbar
""";

hbarEQ1 = True

In [5]:
"""
Generate Spin Operators
""";

System = PyOR.Numerical_MR(Slist1,hbarEQ1)

"""
Sx, Sy and Sz Operators
""";
Sx,Sy,Sz = System.SpinOperator()

"""
S+ and S- Operators
""";
Sp,Sm = System.PMoperators(Sx,Sy)

## Zeeman Hamiltonian in Lab Frame

Let generate Hamiltonians

In [6]:
"""
Gyromagnetic Ratio
Gamma = [Gyromagnetic Ratio spin 1, Gyromagnetic Ratio spin 1, ...]
""";
Gamma = [System.gammaH1,System.gammaH1]

"""
Define the field of the spectromter, B0 in Tesla.
"""
B0 = 9.4

"""
Define the chemical Shift of individual spins
Offset = [chemical Shift spin 1, chemical Shift spin 1, ..]
"""
Offset = [10,20] # Offset frequency in Hz

"""
Function "LarmorF" give the list Larmor frequencies of individual spins in lab frame
"""
LarmorF = System.LarmorFrequency(Gamma,B0,Offset)

Larmor Frequency in MHz:  [-400.22802765 -400.22803765]


In [7]:
Hz = System.Zeeman(LarmorF,Sz)

In [8]:
"""
Now lets see how to get the eigen vectors of the Zeman Hamiltonian (lab frame)
""";
B_Z = System.ZBasis_H(Hz)

<IPython.core.display.Latex object>

In [9]:
"""
Singlet Triplet Basis
""";
B_ST = System.STBasis(B_Z)

<IPython.core.display.Latex object>

In [10]:
"""
Basis Transformation
""";
U = System.Transform_StateBasis(B_Z,B_ST)
Matrix(U)

Matrix([
[1.0,                 0,                  0,   0],
[  0, 0.707106781186547,  0.707106781186547,   0],
[  0,                 0,                  0, 1.0],
[  0, 0.707106781186547, -0.707106781186547,   0]])

## Zeeman Halitonian in Rotating Frame

In [11]:
OmegaRF = [-System.gammaH1*B0,-System.gammaH1*B0]
Hzr = System.Zeeman_RotFrame(LarmorF,Sz,OmegaRF)
Matrix(Hzr/(2.0*np.pi))

Matrix([
[-15.0000000000126,                0,                 0,                0],
[                0, 5.00000002530119,                 0,                0],
[                0,                0, -5.00000002530119,                0],
[                0,                0,                 0, 15.0000000000126]])

## J Coupling Hamiltonian

In [12]:
'''
Define J couplings between individual spins
'''
Jlist = np.zeros((len(Slist1),len(Slist1)))
Jlist[0][1] = 5.0
Hj = System.Jcoupling(Jlist,Sx,Sy,Sz)
Matrix(Hj/(2.0*np.pi))

Matrix([
[1.25,     0,     0,    0],
[   0, -1.25,   2.5,    0],
[   0,   2.5, -1.25,    0],
[   0,     0,     0, 1.25]])

## Basis Transformation of Hamiltonina from Zeeman to Singlet Triplet

In [13]:
"""
J coupling Hamiltonian
""";
Hj_ST = System.Operator_BasisChange(Hj,U)
Matrix(System.Matrix_Tol(Hj_ST,1.0e-10)/(2.0*np.pi))

Matrix([
[1.25,    0,    0,     0],
[   0, 1.25,    0,     0],
[   0,    0, 1.25,     0],
[   0,    0,    0, -3.75]])

In [14]:
"""
Zeeman Hamiltonian Rotating Frame
""";
Hzr_ST = System.Operator_BasisChange(Hzr,U)
Matrix(System.Matrix_Tol(Hzr_ST,1.0e-10)/(2.0*np.pi))

Matrix([
[-15.0000000000126,                0,                0,                0],
[                0,                0,                0, 5.00000002530119],
[                0,                0, 15.0000000000126,                0],
[                0, 5.00000002530119,                0,                0]])

In [15]:
"""
Sz Spin operator
""";

In [16]:
Matrix(Sz[0])

Matrix([
[0.5,   0,    0,    0],
[  0, 0.5,    0,    0],
[  0,   0, -0.5,    0],
[  0,   0,    0, -0.5]])

In [17]:
Sz_ST = System.SpinOperator_BasisChange(Sz,U)
Matrix(System.Matrix_Tol(Sz_ST[0],1.0e-10))

Matrix([
[0.5,   0,    0,   0],
[  0,   0,    0, 0.5],
[  0,   0, -0.5,   0],
[  0, 0.5,    0,   0]])

In [18]:
"""
Sx Spin operator
""";

In [19]:
Matrix(Sx[0])

Matrix([
[  0,   0, 0.5,   0],
[  0,   0,   0, 0.5],
[0.5,   0,   0,   0],
[  0, 0.5,   0,   0]])

In [20]:
Sx_ST = System.SpinOperator_BasisChange(Sx,U)
Matrix(System.Matrix_Tol(Sx_ST[0],1.0e-10))

Matrix([
[                 0, 0.353553390593274,                 0, -0.353553390593274],
[ 0.353553390593274,                 0, 0.353553390593274,                  0],
[                 0, 0.353553390593274,                 0,  0.353553390593274],
[-0.353553390593274,                 0, 0.353553390593274,                  0]])

In [21]:
"""
Sy Spin operator
""";

In [22]:
Matrix(Sy[0])

Matrix([
[    0,     0, -0.5*I,      0],
[    0,     0,      0, -0.5*I],
[0.5*I,     0,      0,      0],
[    0, 0.5*I,      0,      0]])

In [23]:
Sy_ST = System.SpinOperator_BasisChange(Sy,U)
Matrix(System.Matrix_Tol(Sy_ST[0],1.0e-10))

Matrix([
[                   0, -0.353553390593274*I,                    0, 0.353553390593274*I],
[ 0.353553390593274*I,                    0, -0.353553390593274*I,                   0],
[                   0,  0.353553390593274*I,                    0, 0.353553390593274*I],
[-0.353553390593274*I,                    0, -0.353553390593274*I,                   0]])

In [24]:
"""
Construct J Coupling Hamiltonian
"""
Hj_ST = Jlist[0][1] * (Sx_ST[0] @ Sx_ST[1] + Sy_ST[0] @ Sy_ST[1] + Sz_ST[0] @ Sz_ST[1])
Matrix(System.Matrix_Tol(Hj_ST,1.0e-10))

Matrix([
[1.25,    0,    0,     0],
[   0, 1.25,    0,     0],
[   0,    0, 1.25,     0],
[   0,    0,    0, -3.75]])

## Any suggestion? write to me
If you see something is wrong please write to me, so that the PyOR can be error free. 

vineethfrancis.physics@gmail.com